<a href="https://colab.research.google.com/github/dhyaneesh/Sentinel-1-Man-Made-Change-Detection/blob/main/Change_Detection_and_Edge_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import ee
import geemap
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import chi2

# Initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='norse-fragment-435517-g1')

In [28]:
def run_change_detection(roi_geojson, start_date, end_date, significance_level=0.01, glcm_thresholds=None):
    if glcm_thresholds is None:
        glcm_thresholds = {
            'labels_asm': 0.5,
            'labels_contrast': 0.5,
            'labels_corr': 0.5,
            'labels_var': 0.5,
            'labels_idm': 0.5,
            'labels_savg': 0.5,
            'labels_svar': 0.5,
            'labels_sent': 0.5,
            'labels_ent': 0.5,
            'labels_dvar': 0.5,
            'labels_dent': 0.5,
            'labels_imcorr1': 0.5,
            'labels_imcorr2': 0.5,
            'labels_maxcorr': 0.5,
            'labels_diss': 0.5,
            'labels_inertia': 0.5,
            'labels_shade': 0.5,
            'labels_prom': 0.5
        }

    aoi = ee.Geometry.Polygon(roi_geojson['features'][0]['geometry']['coordinates'])

    image_collection = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
                        .filterBounds(aoi)
                        .filterDate(ee.Date(start_date), ee.Date(end_date))
                        .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
                        .map(lambda img: img.set('date', ee.Date(img.date()).format('YYYYMMdd'))))

    clipped_images = image_collection.toList(image_collection.size())
    vv_images = clipped_images.map(lambda img: ee.Image(img).select('VV'))

    k = clipped_images.size().getInfo()
    omnibus_stat = omnibus(vv_images)
    p_value = ee.Image.constant(1).subtract(chi2cdf(omnibus_stat, k - 1))
    change_map = p_value.lt(significance_level)

    object_id = change_map.connectedComponents(connectedness=ee.Kernel.plus(4), maxSize=128)
    glcm_textures = object_id.select('labels').toInt().glcmTexture(size=3)

    man_made_mask = ee.Image.constant(1)
    for feature, threshold in glcm_thresholds.items():
        man_made_mask = man_made_mask.And(glcm_textures.select(feature).gt(threshold))

    man_made_change_map = change_map.And(man_made_mask)

    combined_change_map = ee.Image.constant(0)
    combined_change_map = combined_change_map.where(change_map.And(man_made_mask.Not()), 1)
    combined_change_map = combined_change_map.where(man_made_change_map, 2)

    return {
        'aoi': aoi,
        'combined_change_map': combined_change_map,
        'object_id': object_id
    }

def chi2cdf(chi2_img, df):
    return ee.Image(chi2_img.divide(2)).gammainc(ee.Number(df).divide(2))

def omnibus(im_list, m=4):
    im_list = ee.List(im_list)
    k = im_list.length()
    k_log_k = k.multiply(k.log())
    k_log_k = ee.Image.constant(k_log_k)
    sum_logs = ee.ImageCollection(im_list.map(lambda img: ee.Image(img).log())).reduce(ee.Reducer.sum())
    log_sum = ee.ImageCollection(im_list).reduce(ee.Reducer.sum()).log()
    return k_log_k.add(sum_logs).subtract(log_sum.multiply(k)).multiply(-2*m)

In [29]:
start_date = '2022-01-01'
end_date = '2024-09-27'
significance_level = 1e-200

glcm_thresholds = {
    'labels_asm': 0.4,
    'labels_contrast': 0.6,
    'labels_corr': 0.5,
    'labels_var': 0.7,
    'labels_idm': 0.5,
    'labels_savg': 0.5,
    'labels_svar': 0.5,
    'labels_sent': 0.5,
    'labels_ent': 0.5,
    'labels_dvar': 0.5,
    'labels_dent': 0.5,
    'labels_imcorr1': 0.5,
    'labels_imcorr2': 0.5,
    'labels_maxcorr': 0.5,
    'labels_diss': 0.5,
    'labels_inertia': 0.5,
    'labels_shade': 0.5,
    'labels_prom': 0.5
}

roi_geojson = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            [
              80.48520529483403,
              16.499223030657447
            ],
            [
              80.48520529483403,
              16.482148536458553
            ],
            [
              80.50646914601634,
              16.482148536458553
            ],
            [
              80.50646914601634,
              16.499223030657447
            ],
            [
              80.48520529483403,
              16.499223030657447
            ]
          ]
        ],
        "type": "Polygon"
      },
      "id": 0
    }
  ]
}

result_layers = run_change_detection(roi_geojson, start_date, end_date, significance_level, glcm_thresholds)

Map = geemap.Map()
Map.centerObject(result_layers['aoi'], 10)

Map.addLayer(result_layers['combined_change_map'].updateMask(result_layers['combined_change_map']),
             {'min': 0, 'max': 2, 'palette': ['black', 'blue', 'red']},
             'Change Map (Black: No Change, Blue: Other Changes, Red: Man-Made Changes)')
Map.addLayer(result_layers['object_id'].randomVisualizer(), None, 'Objects')

display(Map)

Map(center=[16.490685926524023, 80.49583722042824], controls=(WidgetControl(options=['position', 'transparent_…